In [2]:
import nltk
nltk.download("punkt")
import numpy as np
import re
import urllib

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vipul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
np.random.seed(1234)